In [9]:
import pandas as pd

In [10]:
flights = pd.read_csv("vuelos.csv")

In [11]:
flights

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,...,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE
0,0.0,53185.0,0.0,8165.0,EK,20392,Emirates,9678A,I,EK,...,Texas,US,United States,74,2019,1,3,17,G,IF
1,0.0,9002.0,0.0,6849.0,EK,20392,Emirates,9678A,I,EK,...,New York,US,United States,22,2019,1,3,14,G,IF
2,0.0,2220750.0,0.0,7247.0,EK,20392,Emirates,9678A,I,EK,...,Illinois,US,United States,41,2019,1,3,15,G,IF
3,0.0,1201490.0,0.0,8165.0,EK,20392,Emirates,9678A,I,EK,...,NaN,AE,United Arab Emirates,678,2019,1,3,17,G,IF
4,0.0,248642.0,0.0,6849.0,EK,20392,Emirates,9678A,I,EK,...,NaN,AE,United Arab Emirates,678,2019,1,3,14,G,IF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321404,96984.0,505308.0,15979.0,1448.0,AS,19930,Alaska Airlines Inc.,06031,D,AS,...,Washington,US,United States,93,2019,3,7,3,F,DU
321405,97011.0,473666.0,55963.0,1448.0,AS,19930,Alaska Airlines Inc.,06031,D,AS,...,Alaska,US,United States,1,2019,2,6,3,F,DU
321406,97098.0,592775.0,18834.0,1448.0,AS,19930,Alaska Airlines Inc.,06031,D,AS,...,Washington,US,United States,93,2019,3,8,3,F,DU
321407,97329.0,210292.0,5431.0,404.0,DL,19790,Delta Air Lines Inc.,01260,D,DL,...,Georgia,US,United States,34,2019,1,3,1,F,DU
